In [1]:
# 連接到 google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
import datetime
import pandas as pd
starttime = datetime.datetime.now()
train = pd.read_csv('/content/drive/MyDrive/wsdm/train_final.csv')
test = pd.read_csv('/content/drive/MyDrive/wsdm/test_final.csv')
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

46


In [ ]:
# 執行時間
# starttime = datetime.datetime.now()
# endtime = datetime.datetime.now()
# print((endtime - starttime).seconds)

In [8]:
test.shape

(2556790, 13)

In [6]:
train.head(2)

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,registered_via,song_length,artist_name,language,time,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,17,7,206471.0,Bastille,52.0,2103.0,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,13,24,9,284584.0,Various Artists,52.0,2301.0,1


In [7]:
test.head(2)

,id,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,registered_via,song_length,artist_name,language,time
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,1,17,7,224130.0,梁文音 (Rachel Liang),3.0,577.0
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,1,17,7,320470.0,林俊傑 (JJ Lin),3.0,577.0


In [3]:
test = test.drop(['Unnamed: 0'],axis=1)
train = train.drop(['Unnamed: 0'],axis=1)

In [4]:
# 把target移到最右邊
train['t']=train['target']
train = train.drop(['target'],axis=1)

In [5]:
train['target']=train['t']
train = train.drop(['t'],axis=1)

In [15]:
# train.dtypes

In [ ]:
# test.dtypes

In [8]:
# 有先改成類別，但還是沒變，再做一次
train['city'] = train['city'].astype(str)
train['registered_via'] = train['registered_via'].astype(str)
train['language'] = train['language'].astype(str)

In [9]:
test['city'] = test['city'].astype(str)
test['registered_via'] = test['registered_via'].astype(str)
test['language'] = test['language'].astype(str)

In [ ]:
# train.dtypes

In [33]:
# test.dtypes

類別轉成數值

In [22]:
for_label_train = train 

In [25]:
# 演算法不能直接跑類別型資料
# 要用 LabelEncoder 或是 dummy 轉換才能算
cols = ['msno', 'song_id', 'source_system_tab', 'source_screen_name','source_type','city', 'registered_via','artist_name', 'language']

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
starttime = datetime.datetime.now()
for i in cols:
    for_label_train[i] = le.fit_transform(for_label_train[i])
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

20


In [28]:
for_label_train.head(5)

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,registered_via,song_length,artist_name,language,time,target
0,8158,74679,1,7,6,0,17,3,206471.0,3277,8,2103.0,1
1,17259,223479,3,8,4,4,24,4,284584.0,31961,8,2301.0,1
2,17259,120758,3,8,4,4,24,4,225396.0,21372,8,2301.0,1
3,17259,23707,3,8,4,4,24,4,255512.0,27440,0,2301.0,1
4,8158,33308,1,7,6,0,17,3,187802.0,4472,8,2103.0,1


In [29]:
for_label_train.corr()

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,registered_via,song_length,artist_name,language,time,target
msno,1.000000,-0.000329,0.011449,-0.002109,-0.002818,0.013243,-0.008395,-0.000822,0.002156,0.003521,-0.006514,-0.005185,0.000982
song_id,-0.000329,1.000000,-0.003903,0.001312,0.005562,-0.000417,-0.000121,-0.000969,-0.010905,-0.021232,0.004091,-0.001099,-0.003742
source_system_tab,0.011449,-0.003903,1.000000,-0.001072,-0.102892,0.036132,0.023360,0.029855,0.024490,0.035023,-0.060251,0.045158,0.031600
source_screen_name,-0.002109,0.001312,-0.001072,1.000000,0.352605,-0.007772,-0.006739,-0.016567,0.024177,0.005287,-0.007192,-0.022757,-0.075587
source_type,-0.002818,0.005562,-0.102892,0.352605,1.000000,-0.011482,0.005215,-0.024732,0.016331,-0.020938,0.003565,-0.037188,-0.126945
city,0.013243,-0.000417,0.036132,-0.007772,-0.011482,1.000000,0.407970,0.062503,0.002747,0.004366,0.003880,0.271254,0.010581
bd,-0.008395,-0.000121,0.023360,-0.006739,0.005215,0.407970,1.000000,0.165606,0.007518,0.007552,-0.025135,0.426809,-0.013135
registered_via,-0.000822,-0.000969,0.029855,-0.016567,-0.024732,0.062503,0.165606,1.000000,0.001972,0.000003,-0.005446,0.432816,0.006385
song_length,0.002156,-0.010905,0.024490,0.024177,0.016331,0.002747,0.007518,0.001972,1.000000,0.113057,-0.189299,0.007793,-0.001809
artist_name,0.003521,-0.021232,0.035023,0.005287,-0.020938,0.004366,0.007552,0.000003,0.113057,1.000000,-0.394049,0.008250,0.012535


In [ ]:
# 正常來說都用dummy的效果會比較好，但全不都用dummy電腦跑不動
# 使用者跟歌曲的種類真的太多了，電腦dummy不動
# 所以用LabelEncoder，來決定哪些用dummy

dataframe concat :
  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

In [44]:
source_system_tab = pd.get_dummies(train['source_system_tab'])
source_screen_name = pd.get_dummies(train['source_screen_name'])
source_type = pd.get_dummies(train['source_type'])
registered_via = pd.get_dummies(train['registered_via'])

In [45]:
print(source_system_tab.shape)
print(source_screen_name.shape)
print(source_type.shape)
print(registered_via.shape)

(7377418, 8)
(7377418, 21)
(7377418, 12)
(7377418, 5)


In [15]:
train.shape

(7377418, 13)

In [46]:
Dummy = pd.concat([source_system_tab,source_screen_name,source_type,registered_via], axis=1)

In [47]:
Dummy.head(2)

,discover,explore,listen with,my library,notification,radio,search,settings,Album more,Artist more,Concert,Discover Chart,Discover Feature,Discover Genre,Discover New,Explore,Local playlist more,My library,My library_Search,NS,Online playlist more,Others profile more,Payment,Radio,Search,Search Home,Search Trends,Self profile more,Unknown,album,artist,listen-with,local-library,local-playlist,my-daily-playlist,online-playlist,radio,song,song-based-playlist,top-hits-for-artist,topic-article-playlist,3,4,7,9,13
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0


In [19]:
train = pd.concat([train,Dummy], axis=1)

In [20]:
train.shape

(7377418, 59)

In [21]:
train = train.drop(['source_system_tab','source_screen_name','source_type','registered_via'],axis=1)

In [22]:
train.shape

(7377418, 55)

In [23]:
cols = ['msno', 'song_id','city','artist_name', 'language']

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
starttime = datetime.datetime.now()
for i in cols:
    train[i] = le.fit_transform(train[i])
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

17


In [ ]:
# train.dtypes

In [26]:
# 把target移到最右邊
train['t']=train['target']
train = train.drop(['target'],axis=1)

In [27]:
train['target']=train['t']
train = train.drop(['t'],axis=1)

In [30]:
# train.dtypes
train.shape

(7377418, 55)

In [31]:
# 執行時間
starttime = datetime.datetime.now()
train.to_csv('/content/drive/MyDrive/wsdm/train_ok.csv')
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

90


test

In [48]:
source_system_tab = pd.get_dummies(test['source_system_tab'])
source_screen_name = pd.get_dummies(test['source_screen_name'])
source_type = pd.get_dummies(test['source_type'])
registered_via = pd.get_dummies(test['registered_via'])

In [49]:
print(source_system_tab.shape)
print(source_screen_name.shape)
print(source_type.shape)
print(registered_via.shape)

(2556790, 8)
(2556790, 23)
(2556790, 12)
(2556790, 6)


In [50]:
Dummy = pd.concat([source_system_tab,source_screen_name,source_type,registered_via], axis=1)

In [51]:
Dummy.head(2)
# 16  People local People global

,discover,explore,listen with,my library,notification,radio,search,settings,Album more,Artist more,Concert,Discover Chart,Discover Feature,Discover Genre,Discover New,Explore,Local playlist more,My library,My library_Search,NS,Online playlist more,Others profile more,Payment,People global,People local,Radio,Search,Search Home,Search Trends,Self profile more,Unknown,album,artist,listen-with,local-library,local-playlist,my-daily-playlist,online-playlist,radio,song,song-based-playlist,top-hits-for-artist,topic-article-playlist,3,4,7,9,13,16
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


下面三個是train沒有的維度，為了一致性，刪掉不採用

In [ ]:
# 2
Dummy[16].value_counts()

In [ ]:
# 13
Dummy['People local'].value_counts()

In [ ]:
# 1
Dummy['People global'].value_counts()

In [35]:
test.shape

(2556790, 13)

In [37]:
Dummy.shape

(2556790, 49)

In [56]:
test = pd.concat([test,Dummy], axis=1)

In [57]:
test.shape

(2556790, 63)

In [58]:
test = test.drop([16,'People local','People local'],axis=1)

In [59]:
test = test.drop(['source_system_tab','source_screen_name','source_type','registered_via'],axis=1)

In [60]:
test.shape

(2556790, 57)

In [61]:
cols = ['msno', 'song_id','city','artist_name', 'language']

In [62]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
starttime = datetime.datetime.now()
for i in cols:
    test[i] = le.fit_transform(test[i])
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

5


In [64]:
# 執行時間
starttime = datetime.datetime.now()
test.to_csv('/content/drive/MyDrive/wsdm/test_ok.csv')
endtime = datetime.datetime.now()
print((endtime - starttime).seconds)

33
